In [1]:
!pip install nltk -q
!pip install datasets -q

     |████████████████████████████████| 441 kB 5.3 MB/s 
     |████████████████████████████████| 163 kB 63.2 MB/s 
     |████████████████████████████████| 212 kB 64.9 MB/s 
     |████████████████████████████████| 115 kB 67.0 MB/s 
     |████████████████████████████████| 95 kB 4.7 MB/s 
     |████████████████████████████████| 127 kB 53.6 MB/s 
     |████████████████████████████████| 115 kB 46.3 MB/s 


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from datasets import load_dataset
datasets = load_dataset("tweet_eval", "emoji")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [7]:
tr_text = datasets["train"]["text"]
tr_label = datasets["train"]["label"]

ts_text = datasets["test"]["text"]
ts_label = datasets["test"]["label"]

v_text = datasets["validation"]["text"]
v_label = datasets["validation"]["label"]

In [9]:
import pandas as pd
df = pd.DataFrame()

In [10]:
df['text'] = tr_text + ts_text + v_text
df['label'] = tr_label + ts_label + v_label
df.head(10)


,text,label
0,Sunday afternoon walking through Venice in the...,12
1,Time for some BBQ and whiskey libations. Chomp...,19
2,Love love love all these people ️ ️ ️ #friends...,0
3,"️ ️ ️ ️ @ Toys""R""Us",0
4,Man these are the funniest kids ever!! That fa...,2
5,"#sandiego @ San Diego, California",11
6,My little ️ ️ ️ ️ ️ #ObsessedWithMyDog @ Cafe ...,0
7,"More #tinyepic things #tinyepicwestern, this o...",19
8,Last night ️ @ Omnia Night Club At Caesars Palace,0
9,friendship at its finest. ....#pixar #toystory...,7


In [13]:
df['label'].value_counts()

0     21058
1     10252
2      9751
4      6105
7      5293
3      4956
6      4278
18     4056
5      3983
10     3573
17     3063
11     3038
8      3013
9      2737
14     2659
16     2640
12     2640
19     2332
15     2326
13     2247
Name: label, dtype: int64

In [14]:
df2 = df[df["label"].isin([14,17])]
df2.head()

,text,label
31,#Lunch @user best #Ribs in #SanDiego (at @user...,14
35,Surprised everyone by actually surviving a wee...,14
40,Stay classy #SanDiego (@ San Diego Internation...,14
74,@user hi from Tijuana B.C. See you soon,14
77,Birthday cheers with my favorite people!!#26ag...,14


In [15]:
df2['label'].value_counts()

17    3063
14    2659
Name: label, dtype: int64

In [16]:
sample = df2['text'].iloc[0]
print(sample)

#Lunch @user best #Ribs in #SanDiego (at @user in San Diego, CA)


In [18]:
print(str(sample).replace("@", ""))

#Lunch user best #Ribs in #SanDiego (at user in San Diego, CA)


In [19]:
print(str(sample).replace("#", ""))

Lunch @user best Ribs in SanDiego (at @user in San Diego, CA)


In [21]:
df2['text'] = df2['text'].str.replace("@", "")
df2['text'] = df2['text'].str.replace("#", "")
df2['text'] = df2['text'].str.replace("\d+", "")
df2['text'] = df2['text'].str.replace("http\S+|www.\S+", "")
df2['text'] = df2['text'].str.lower()
df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/us

,text,label
31,lunch user best ribs in sandiego (at user in s...,14
35,surprised everyone by actually surviving a wee...,14
40,stay classy sandiego ( san diego international...,14
74,user hi from tijuana b.c. see you soon,14
77,birthday cheers with my favorite people!!again...,14


In [22]:
df2.shape

(5722, 2)

In [23]:
#bow
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
bow.fit(df2['text'])
bow = bow.transform(df2['text'])

print(bow.shape)

(5722, 11801)


In [26]:
from sklearn.model_selection import train_test_split as ts


In [28]:
#modelling bow
X_train, X_test, y_train, y_test = ts(bow, df2['label'], test_size=0.2, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test, y_pred))

0.6724890829694323


In [42]:
import nltk
from nltk.corpus import stopwords
stop =stopwords.words('english')

#tf idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    stop_words=stop,
    ngram_range=(1,1)

)
tfidf.fit(df2['text'])
tfidf = tfidf.transform(df2['text'])
print(tfidf.shape)


(5722, 11668)


In [39]:
# modelling TfIdf
from sklearn.model_selection import train_test_split as ts
X_train, X_test, y_train, y_test = ts(tfidf, df2['label'], test_size = 0.2, random_state=42)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test, y_pred))


0.4707423580786026


In [43]:
# modelling TfIdf NB
from sklearn.model_selection import train_test_split as ts
X_train, X_test, y_train, y_test = ts(tfidf, df2['label'], test_size = 0.2, random_state=42)

from sklearn.naive_bayes import MultinomialNB
knn = MultinomialNB()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test, y_pred))

0.8078602620087336
